# データハンドリング

## この章で扱うもの

- 千葉市の過去3年間の気象データ
- 千葉市の過去3年間のインフルエンザ発見件数のデータ
- 上記のデータを組合せて、この次の章で使用する可視化や機械学習で使用しやすいデータにする

### 気象データ

気象データは気象庁のWebサイトから様々なデータが取得可能です。
今回は、過去3年間の千葉市をターゲットにしているので、必要な項目を選択しCSVで入手します。


### インフルエンザ発見件数

千葉市のオープンデータに、「インフルエンザの週毎の定点当たり報告数」というものがあり、それを用いる


## この章で取り扱う手順

- データの入手(2箇所から、CSVデータ)
- データの概要確認
- データの結合
- データの加工
- 欠損値処理


# 必要なライブラリのインポート

In [1]:
import numpy as np
import pandas as pd

# データの入手・概要を確認・加工

## 気象データ

気象庁の過去の気象データ
http://www.data.jma.go.jp/gmd/risk/obsdl/index.php

In [2]:
df_weather = pd.read_csv("data/気象庁千葉.csv", encoding='shift_jis')

pandas の read_csv関数で、CSVデータを読込。読み込みの際に文字エンコーディングを指定している

In [3]:
df_weather.head()

ダウンロードした時刻：2017/06/17 12:09:19
NaN      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉      千葉        千葉        千葉        千葉          千葉          千葉          千葉         千葉         千葉         千葉         千葉       千葉       千葉       千葉       千葉        千葉                                    千葉
年月日      平均気温(℃) 平均気温(℃) 平均気温(℃) 最高気温(℃) 最高気温(℃) 最高気温(℃) 最低気温(℃) 最低気温(℃) 最低気温(℃) 平均湿度(％) 平均湿度(％) 平均湿度(％) 最小相対湿度(％) 最小相対湿度(％) 最小相対湿度(％) 平均現地気圧(hPa) 平均現地気圧(hPa) 平均現地気圧(hPa) 降水量の合計(mm) 降水量の合計(mm) 降水量の合計(mm) 降水量の合計(mm) 日照時間(時間) 日照時間(時間) 日照時間(時間) 日照時間(時間) 平均風速(m/s) 平均風速(m/s)                      平均風速(m/s)
NaN      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN       NaN       NaN       NaN         NaN         NaN         NaN        NaN        NaN        NaN        NaN      NaN      NaN      NaN      NaN       NaN                                  NaN
                 品質情報    均質番号    NaN     品質情報    均質番号    NaN     品質情報    均質番号    NaN     品質情報    均質番号    NaN       品質情報      均質番号      NaN         品質情報        均質番号        NaN        現象なし情報     品質情報       均質番号       NaN      現象なし情報   品質情報     均質番号     NaN       品質情報                                均質番号
2014/1/1 9.8     8       1       13.7    8       1       3.9     8       1       54      8       1       37        8         1         1005.3      8           1           0          1          8          1          9.2      0        8        1        5.3       8                                      1

headメソッドで、データの先頭を確認。うまく取り込みているようです。

In [4]:
df_weather.shape

(1220, 1)

データの行と列の数を確認。なぜか、1列になっています。

In [5]:
f = open("data/気象庁千葉.csv", encoding="shift_jis")
for l in f.readlines()[:10]:
    print(l)

ダウンロードした時刻：2017/06/17 12:09:19



,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉,千葉

年月日,平均気温(℃),平均気温(℃),平均気温(℃),最高気温(℃),最高気温(℃),最高気温(℃),最低気温(℃),最低気温(℃),最低気温(℃),平均湿度(％),平均湿度(％),平均湿度(％),最小相対湿度(％),最小相対湿度(％),最小相対湿度(％),平均現地気圧(hPa),平均現地気圧(hPa),平均現地気圧(hPa),降水量の合計(mm),降水量の合計(mm),降水量の合計(mm),降水量の合計(mm),日照時間(時間),日照時間(時間),日照時間(時間),日照時間(時間),平均風速(m/s),平均風速(m/s),平均風速(m/s)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,品質情報,均質番号,,現象なし情報,品質情報,均質番号,,現象なし情報,品質情報,均質番号,,品質情報,均質番号

2014/1/1,9.8,8,1,13.7,8,1,3.9,8,1,54,8,1,37,8,1,1005.3,8,1,0,1,8,1,9.2,0,8,1,5.3,8,1

2014/1/2,8.0,8,1,12.9,8,1,4.4,8,1,41,8,1,26,8,1,1011.3,8,1,0,1,8,1,9.1,0,8,1,3.0,8,1

2014/1/3,5.9,8,1,9.9,8,1,2.7,8,1,43,8,1,32,8,1,1014.9,8,1,0,1,8,1,4.1,0,8,1,1.6,8,1

2014/1/4,6.7,8,1,11.5,8,1,2.1,8,1,47,8,1,29,8,1,1009.5,8,1,0,1,8,1,5.9,0,8,1,2.4,8,1



元のCSVをテキストファイルとして読み込んで表示してみます。
行と列の情報以外に１行目にデータ以外の情報が入っていました。

In [6]:
df_weather = pd.read_csv("data/気象庁千葉.csv", encoding='shift_jis', skiprows=3)

気象データのCSVを再度読み込みます。今回は、３行目まで読み込み無視するように宣言しました。

In [7]:
df_weather.shape

(1218, 30)

問題なく取り込めているようです。

In [8]:
df_weather.head()

,年月日,平均気温(℃),平均気温(℃).1,平均気温(℃).2,最高気温(℃),最高気温(℃).1,最高気温(℃).2,最低気温(℃),最低気温(℃).1,最低気温(℃).2,...,降水量の合計(mm).1,降水量の合計(mm).2,降水量の合計(mm).3,日照時間(時間),日照時間(時間).1,日照時間(時間).2,日照時間(時間).3,平均風速(m/s),平均風速(m/s).1,平均風速(m/s).2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,品質情報,均質番号,NaN,品質情報,均質番号,NaN,品質情報,均質番号,...,現象なし情報,品質情報,均質番号,NaN,現象なし情報,品質情報,均質番号,NaN,品質情報,均質番号
2,2014/1/1,9.8,8,1,13.7,8,1,3.9,8,1,...,1,8,1,9.2,0,8,1,5.3,8,1
3,2014/1/2,8.0,8,1,12.9,8,1,4.4,8,1,...,1,8,1,9.1,0,8,1,3.0,8,1
4,2014/1/3,5.9,8,1,9.9,8,1,2.7,8,1,...,1,8,1,4.1,0,8,1,1.6,8,1


In [9]:
df_weather.tail()

,年月日,平均気温(℃),平均気温(℃).1,平均気温(℃).2,最高気温(℃),最高気温(℃).1,最高気温(℃).2,最低気温(℃),最低気温(℃).1,最低気温(℃).2,...,降水量の合計(mm).1,降水量の合計(mm).2,降水量の合計(mm).3,日照時間(時間),日照時間(時間).1,日照時間(時間).2,日照時間(時間).3,平均風速(m/s),平均風速(m/s).1,平均風速(m/s).2
1213,2017/4/26,17.0,8,1,19.7,8,1,14.2,8,1,...,1,8,1,0.2,0,8,1,6.5,8,1
1214,2017/4/27,14.7,8,1,17.6,8,1,11.6,8,1,...,0,8,1,2.6,0,8,1,4.5,8,1
1215,2017/4/28,16.0,8,1,20.1,8,1,12.7,8,1,...,1,8,1,7.4,0,8,1,2.7,8,1
1216,2017/4/29,16.3,8,1,21.9,8,1,11.8,8,1,...,1,8,1,12.1,0,8,1,3.5,8,1
1217,2017/4/30,18.0,8,1,23.2,8,1,12.1,8,1,...,1,8,1,12.6,0,8,1,3.7,8,1


.head()メソッドと.tail()メソッドを使い、データの概要を見てみました。データとしては良さそうです。

In [10]:
df_weather.iloc[1, :]

年月日                 NaN
平均気温(℃)             NaN
平均気温(℃).1          品質情報
平均気温(℃).2          均質番号
最高気温(℃)             NaN
最高気温(℃).1          品質情報
最高気温(℃).2          均質番号
最低気温(℃)             NaN
最低気温(℃).1          品質情報
最低気温(℃).2          均質番号
平均湿度(％)             NaN
平均湿度(％).1          品質情報
平均湿度(％).2          均質番号
最小相対湿度(％)           NaN
最小相対湿度(％).1        品質情報
最小相対湿度(％).2        均質番号
平均現地気圧(hPa)         NaN
平均現地気圧(hPa).1      品質情報
平均現地気圧(hPa).2      均質番号
降水量の合計(mm)          NaN
降水量の合計(mm).1     現象なし情報
降水量の合計(mm).2       品質情報
降水量の合計(mm).3       均質番号
日照時間(時間)            NaN
日照時間(時間).1       現象なし情報
日照時間(時間).2         品質情報
日照時間(時間).3         均質番号
平均風速(m/s)           NaN
平均風速(m/s).1        品質情報
平均風速(m/s).2        均質番号
Name: 1, dtype: object

In [11]:
df_weather.iloc[1, :].notnull()

年月日              False
平均気温(℃)          False
平均気温(℃).1         True
平均気温(℃).2         True
最高気温(℃)          False
最高気温(℃).1         True
最高気温(℃).2         True
最低気温(℃)          False
最低気温(℃).1         True
最低気温(℃).2         True
平均湿度(％)          False
平均湿度(％).1         True
平均湿度(％).2         True
最小相対湿度(％)        False
最小相対湿度(％).1       True
最小相対湿度(％).2       True
平均現地気圧(hPa)      False
平均現地気圧(hPa).1     True
平均現地気圧(hPa).2     True
降水量の合計(mm)       False
降水量の合計(mm).1      True
降水量の合計(mm).2      True
降水量の合計(mm).3      True
日照時間(時間)         False
日照時間(時間).1        True
日照時間(時間).2        True
日照時間(時間).3        True
平均風速(m/s)        False
平均風速(m/s).1       True
平均風速(m/s).2       True
Name: 1, dtype: bool

In [12]:
[i for i, t in enumerate(df_weather.iloc[1, :].isnull()) if t]

[0, 1, 4, 7, 10, 13, 16, 19, 23, 27]

In [13]:
df_weather_temp = df_weather.iloc[:, [i for i, t in enumerate(df_weather.iloc[1, :].isnull()) if t]]

ここではリスト内包表記でカラムの位置を取得していますが以下でも同じ動作になります。(pandasの機能を用いた方法)

In [14]:
df_weather = df_weather[df_weather.columns[df_weather.iloc[1].isnull()]]

In [15]:
df_weather.head()

,年月日,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014/1/1,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
3,2014/1/2,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
4,2014/1/3,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6


In [16]:
df_weather.tail()

,年月日,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
1213,2017/4/26,17.0,19.7,14.2,69.0,53.0,1014.3,0.0,0.2,6.5
1214,2017/4/27,14.7,17.6,11.6,69.0,45.0,1004.9,6.5,2.6,4.5
1215,2017/4/28,16.0,20.1,12.7,62.0,31.0,1006.2,0.0,7.4,2.7
1216,2017/4/29,16.3,21.9,11.8,64.0,34.0,1005.7,0.0,12.1,3.5
1217,2017/4/30,18.0,23.2,12.1,57.0,31.0,1009.2,0.0,12.6,3.7


In [17]:
df_weather.shape

(1218, 10)

In [18]:
df_weather = df_weather.drop([0, 1])

In [19]:
df_weather['年月日'] = pd.to_datetime(df_weather["年月日"])

In [20]:
df_weather = df_weather.set_index('年月日')

In [21]:
df_weather.head()

,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
年月日,,,,,,,,,
2014-01-01,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
2014-01-02,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
2014-01-03,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6
2014-01-04,6.7,11.5,2.1,47.0,29.0,1009.5,0.0,5.9,2.4
2014-01-05,4.4,6.9,2.3,40.0,28.0,1016.6,0.0,1.1,2.5


In [22]:
df_weather.tail()

,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
年月日,,,,,,,,,
2017-04-26,17.0,19.7,14.2,69.0,53.0,1014.3,0.0,0.2,6.5
2017-04-27,14.7,17.6,11.6,69.0,45.0,1004.9,6.5,2.6,4.5
2017-04-28,16.0,20.1,12.7,62.0,31.0,1006.2,0.0,7.4,2.7
2017-04-29,16.3,21.9,11.8,64.0,34.0,1005.7,0.0,12.1,3.5
2017-04-30,18.0,23.2,12.1,57.0,31.0,1009.2,0.0,12.6,3.7


In [23]:
df_weather.dtypes

平均気温(℃)        float64
最高気温(℃)        float64
最低気温(℃)        float64
平均湿度(％)        float64
最小相対湿度(％)      float64
平均現地気圧(hPa)    float64
降水量の合計(mm)     float64
日照時間(時間)       float64
平均風速(m/s)      float64
dtype: object

In [24]:
df_weather.to_pickle("df_weather.db")

## 千葉市のインフルエンザの状況

千葉県のオープンデータ
http://www.city.chiba.jp/opendata/index.php

- インフルエンザの週毎の定点当たり報告数
- ライセンス: CC-BY
- 2014年、2015年、2016年

In [25]:
!ls data/

201412influenza.csv     201612influenza.csv     気象庁千葉.csv
201512influenza.csv     201712influenza.csv     気象庁千葉2017.csv


In [26]:
df_2014 = pd.read_csv("data/201412influenza.csv", encoding="shift_jis")

In [27]:
df_2014.head()

,2014年,インフルエンザ,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52
0,週数,第1週,第2週,第3週,第4週,第5週,第6週,第7週,第8週,第9週,...,第43週,第44週,第45週,第46週,第47週,第48週,第49週,第50週,第51週,第52週
1,市全体（報告数/定点数）,0.178571429,2.814814815,9.75,25.32142857,31.32142857,33.25,26.21428571,25.53571429,30.60714286,...,0.285714286,0.571428571,0.571428571,1.107142857,3.111111111,5.357142857,7.964285714,12.75,20.21428571,31.08
2,中央区（報告数/定点数）,0.75,7,22.25,47.25,63,54,34,35.25,44.5,...,0,0,0,0.6,2.5,7,7.8,15,27.6,40.4
3,花見川区（報告数/定点数）,0,1,3.75,15.5,17.25,20.25,17,16.5,17,...,0,0.5,0.75,1.25,1.5,2.5,3.75,4.75,7,14.5
4,稲毛区（報告数/定点数）,0.2,2.8,12.8,21.4,27.8,37.4,29.2,24.8,25.2,...,0,0,0,0.8,0.4,1.8,3.8,7.4,17.2,21.2


In [28]:
df_2014 = pd.DataFrame(df_2014.iloc[1, 1:])

In [29]:
df_2014.columns = ['報告数']

In [30]:
df_2014.head()

,報告数
インフルエンザ,0.178571429
Unnamed: 2,2.814814815
Unnamed: 3,9.75
Unnamed: 4,25.32142857
Unnamed: 5,31.32142857


In [31]:
df_2014.tail()

,報告数
Unnamed: 48,5.357142857
Unnamed: 49,7.964285714
Unnamed: 50,12.75
Unnamed: 51,20.21428571
Unnamed: 52,31.08


In [32]:
date_index = pd.date_range(start="2013-12-30", periods=52, freq="W-MON")

In [33]:
date_index

DatetimeIndex(['2013-12-30', '2014-01-06', '2014-01-13', '2014-01-20',
               '2014-01-27', '2014-02-03', '2014-02-10', '2014-02-17',
               '2014-02-24', '2014-03-03', '2014-03-10', '2014-03-17',
               '2014-03-24', '2014-03-31', '2014-04-07', '2014-04-14',
               '2014-04-21', '2014-04-28', '2014-05-05', '2014-05-12',
               '2014-05-19', '2014-05-26', '2014-06-02', '2014-06-09',
               '2014-06-16', '2014-06-23', '2014-06-30', '2014-07-07',
               '2014-07-14', '2014-07-21', '2014-07-28', '2014-08-04',
               '2014-08-11', '2014-08-18', '2014-08-25', '2014-09-01',
               '2014-09-08', '2014-09-15', '2014-09-22', '2014-09-29',
               '2014-10-06', '2014-10-13', '2014-10-20', '2014-10-27',
               '2014-11-03', '2014-11-10', '2014-11-17', '2014-11-24',
               '2014-12-01', '2014-12-08', '2014-12-15', '2014-12-22'],
              dtype='datetime64[ns]', freq='W-MON')

In [34]:
df_2014.index = date_index

In [35]:
df_2014.head()

,報告数
2013-12-30,0.178571429
2014-01-06,2.814814815
2014-01-13,9.75
2014-01-20,25.32142857
2014-01-27,31.32142857


In [36]:
df_2014.tail()

,報告数
2014-11-24,5.357142857
2014-12-01,7.964285714
2014-12-08,12.75
2014-12-15,20.21428571
2014-12-22,31.08


In [37]:
df_2015 = pd.read_csv("data/201512influenza.csv", encoding="shift_jis")

In [38]:
df_2015 = pd.DataFrame(df_2015.iloc[1, 1:])
df_2015.columns = ["報告数"]
date_index_2015 = pd.date_range(start="2014-12-29", periods=53, freq="W-MON")
df_2015.index =date_index_2015
df_2015.head()

,報告数
2014-12-29,4.038461538
2015-01-05,25.77777778
2015-01-12,27.92857143
2015-01-19,31.71428571
2015-01-26,27.21428571


In [39]:
df_2015.tail()

,報告数
2015-11-30,0.071428571
2015-12-07,0.107142857
2015-12-14,0.071428571
2015-12-21,0.148148148
2015-12-28,0


In [40]:
df_2016 = pd.read_csv("data/201612influenza.csv", encoding='shift_jis')

In [41]:
df_2016 = pd.DataFrame(df_2016.iloc[1, 1:])
df_2016.columns = ["報告数"]
date_index_2016 = pd.date_range(start="2016-1-4", periods=53, freq="W-MON")
df_2016.index =date_index_2016
df_2016.head()

,報告数
2016-01-04,1.444444444
2016-01-11,5.857142857
2016-01-18,15.67857143
2016-01-25,30.17857143
2016-02-01,46.21428571


In [42]:
df_2016.tail()

,報告数
2016-12-05,3.857142857
2016-12-12,4.178571429
2016-12-19,6.892857143
2016-12-26,4.434782609
2017-01-02,NaN


## 取り込めたデータを再確認

いずれも、千葉市の

- df_weather:  ２０１４年１月２０１７年４月までの気象データフレーム
- ser_2014:  ２０１４年１月から２０１４年１２月までのインフルエンザの報告件数シリーズ
- ser_2015:  ２０１5年１月から２０１5年１２月までのインフルエンザの報告件数データ
- ser_2016:  ２０１6年１月から２０１6年１２月までのインフルエンザの報告件数データ

In [43]:
df_weather.head()

,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
年月日,,,,,,,,,
2014-01-01,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
2014-01-02,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
2014-01-03,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6
2014-01-04,6.7,11.5,2.1,47.0,29.0,1009.5,0.0,5.9,2.4
2014-01-05,4.4,6.9,2.3,40.0,28.0,1016.6,0.0,1.1,2.5


In [44]:
df_2014.head()

,報告数
2013-12-30,0.178571429
2014-01-06,2.814814815
2014-01-13,9.75
2014-01-20,25.32142857
2014-01-27,31.32142857


In [45]:
df_2015.head()

,報告数
2014-12-29,4.038461538
2015-01-05,25.77777778
2015-01-12,27.92857143
2015-01-19,31.71428571
2015-01-26,27.21428571


In [46]:
df_2016.head()

,報告数
2016-01-04,1.444444444
2016-01-11,5.857142857
2016-01-18,15.67857143
2016-01-25,30.17857143
2016-02-01,46.21428571


# データの結合と欠損値の処理

上記のデータを一つのデータフレームにまとめていきます。
必要に応じて欠損値も処理していきます。

## インフルエンザの報告件数を3年分まとめる

In [47]:
df_influ = pd.concat([df_2014, df_2015, df_2016])

In [48]:
df_influ.shape

(158, 1)

In [49]:
df_influ.head()

,報告数
2013-12-30,0.178571429
2014-01-06,2.814814815
2014-01-13,9.75
2014-01-20,25.32142857
2014-01-27,31.32142857


In [50]:
df_influ.tail()

,報告数
2016-12-05,3.857142857
2016-12-12,4.178571429
2016-12-19,6.892857143
2016-12-26,4.434782609
2017-01-02,NaN


In [51]:
df_influ = df_influ.dropna()

In [52]:
df_influ.shape

(157, 1)

In [53]:
df_infｌu = pd.DataFrame(df_influ.astype(np.float32))

In [54]:
df_influ.head()

,報告数
2013-12-30,0.178571
2014-01-06,2.814815
2014-01-13,9.750000
2014-01-20,25.321428
2014-01-27,31.321428


In [55]:
df_influ.dtypes

報告数    float32
dtype: object

In [56]:
df_influ.to_pickle("df_influ.db")

## 気象データとつなげる

In [57]:
df = pd.concat([df_influ, df_weather], axis=1)

In [58]:
df.shape

(1217, 10)

In [59]:
df.head()

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2013-12-30,0.178571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-01,NaN,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
2014-01-02,NaN,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
2014-01-03,NaN,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6
2014-01-04,NaN,6.7,11.5,2.1,47.0,29.0,1009.5,0.0,5.9,2.4


In [60]:
df.tail(40)

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2017-03-22,NaN,10.5,15.0,6.6,35.0,19.0,1010.8,0.0,9.7,5.5
2017-03-23,NaN,10.0,14.1,6.5,38.0,25.0,1015.2,0.0,5.8,2.3
2017-03-24,NaN,8.7,13.8,5.6,38.0,15.0,1015.3,0.0,4.1,4.0
2017-03-25,NaN,7.5,12.3,3.7,38.0,23.0,1019.8,0.0,10.4,3.2
2017-03-26,NaN,7.5,9.6,5.9,81.0,50.0,1016.9,7.0,0.1,3.0
2017-03-27,NaN,5.1,7.1,2.3,84.0,70.0,1008.8,30.0,0.6,4.9
2017-03-28,NaN,8.3,13.7,3.8,61.0,25.0,1017.9,1.0,9.1,3.4
2017-03-29,NaN,9.4,14.1,5.0,65.0,41.0,1021.6,0.0,6.8,2.7
2017-03-30,NaN,11.9,16.0,6.4,63.0,40.0,1017.2,0.0,9.6,2.6
2017-03-31,NaN,9.1,13.1,5.8,57.0,34.0,1016.5,5.0,1.8,3.0


# 欠損値の処理

In [61]:
df = df.loc["2014-01-01":"2016-12-31", :]

In [62]:
df.shape

(1096, 10)

In [63]:
df.head(10)

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2014-01-01,NaN,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
2014-01-02,NaN,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
2014-01-03,NaN,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6
2014-01-04,NaN,6.7,11.5,2.1,47.0,29.0,1009.5,0.0,5.9,2.4
2014-01-05,NaN,4.4,6.9,2.3,40.0,28.0,1016.6,0.0,1.1,2.5
2014-01-06,2.814815,4.9,10.4,0.8,36.0,19.0,1021.4,0.0,9.4,4.5
2014-01-07,NaN,5.0,8.7,0.2,46.0,30.0,1025.4,0.0,6.5,2.0
2014-01-08,NaN,7.8,13.6,2.5,66.0,39.0,1014.4,18.0,2.5,2.5
2014-01-09,NaN,6.5,9.7,3.1,77.0,39.0,1007.1,10.0,0.6,3.1
2014-01-10,NaN,3.1,6.5,0.8,31.0,17.0,1019.9,0.0,3.3,3.6


In [64]:
df.tail()

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2016-12-27,NaN,13.0,19.2,5.3,81.0,72.0,1002.4,5.0,0.1,7.3
2016-12-28,NaN,6.8,10.2,3.8,40.0,25.0,1018.4,0.5,9.4,4.2
2016-12-29,NaN,5.5,13.2,1.8,51.0,44.0,1017.0,0.0,5.3,2.3
2016-12-30,NaN,6.3,10.2,2.6,35.0,20.0,1022.0,0.0,9.3,4.3
2016-12-31,NaN,6.8,11.0,3.0,44.0,27.0,1023.1,0.0,9.2,1.6


In [65]:
df["報告数"] = df["報告数"].fillna(method='ffill').astype(np.float32)

In [66]:
df.head(10)

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2014-01-01,NaN,9.8,13.7,3.9,54.0,37.0,1005.3,0.0,9.2,5.3
2014-01-02,NaN,8.0,12.9,4.4,41.0,26.0,1011.3,0.0,9.1,3.0
2014-01-03,NaN,5.9,9.9,2.7,43.0,32.0,1014.9,0.0,4.1,1.6
2014-01-04,NaN,6.7,11.5,2.1,47.0,29.0,1009.5,0.0,5.9,2.4
2014-01-05,NaN,4.4,6.9,2.3,40.0,28.0,1016.6,0.0,1.1,2.5
2014-01-06,2.814815,4.9,10.4,0.8,36.0,19.0,1021.4,0.0,9.4,4.5
2014-01-07,2.814815,5.0,8.7,0.2,46.0,30.0,1025.4,0.0,6.5,2.0
2014-01-08,2.814815,7.8,13.6,2.5,66.0,39.0,1014.4,18.0,2.5,2.5
2014-01-09,2.814815,6.5,9.7,3.1,77.0,39.0,1007.1,10.0,0.6,3.1
2014-01-10,2.814815,3.1,6.5,0.8,31.0,17.0,1019.9,0.0,3.3,3.6


In [67]:
df.tail(10)

,報告数,平均気温(℃),最高気温(℃),最低気温(℃),平均湿度(％),最小相対湿度(％),平均現地気圧(hPa),降水量の合計(mm),日照時間(時間),平均風速(m/s)
2016-12-22,6.892857,16.4,21.7,9.1,76.0,60.0,1015.7,4.0,2.1,5.7
2016-12-23,6.892857,15.8,20.9,9.5,53.0,31.0,1000.1,6.5,2.6,7.5
2016-12-24,6.892857,9.3,12.7,6.2,37.0,24.0,1018.9,0.0,8.6,5.0
2016-12-25,6.892857,7.7,11.2,5.5,45.0,33.0,1028.9,0.0,4.6,2.2
2016-12-26,4.434783,8.3,14.4,3.7,56.0,41.0,1022.8,0.0,2.8,1.9
2016-12-27,4.434783,13.0,19.2,5.3,81.0,72.0,1002.4,5.0,0.1,7.3
2016-12-28,4.434783,6.8,10.2,3.8,40.0,25.0,1018.4,0.5,9.4,4.2
2016-12-29,4.434783,5.5,13.2,1.8,51.0,44.0,1017.0,0.0,5.3,2.3
2016-12-30,4.434783,6.3,10.2,2.6,35.0,20.0,1022.0,0.0,9.3,4.3
2016-12-31,4.434783,6.8,11.0,3.0,44.0,27.0,1023.1,0.0,9.2,1.6
